In [1]:
import os.path

import numpy as np
import pandas as pd
from scipy import linalg
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder

import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

from joblib import Parallel, delayed
import timeit

import matplotlib.pyplot as plt
%matplotlib inline

# # --- Import our Code ---# #
#import emachine as EM
from direct_info import direct_info

# import data processing and general DCA_ER tools
from data_processing import data_processing
import ecc_tools as tools
from pathlib import Path
np.random.seed(1)


def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [2]:
data_path = Path('/home', 'eclay','Pfam-A.full')
data_path = Path('Pfam-A.full')
DCA_ER_dir = Path('/home/evan/PycharmProjects/DCA_ER/')



pfam_id = 'PF00023'
pfam_id = 'PF00011'
pfam_id = 'PF00186'

n_jobs = 26
create_new = True


if 0:
    DCA_ER_dir = '/home/eclay/DCA_ER/'
    msa_npy_file = '/home/eclay/Pfam-A.full/%s/msa.npy' % pfam_id # Hurricane Location
    msa_fa_file  = '/home/eclay/Pfam-A.full/%s/msa.fa' % pfam_id # Hurricane Location
    pdb_ref_file = '/home/eclay/Pfam-A.full/%s/pdb_refs.npy' % pfam_id # Hurricane Location
if 1:
    DCA_ER_dir = '/home/evan/DCA_ER/'
    msa_npy_file = '/home/evan/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/home/evan/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/home/evan/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir
if 0:
    DCA_ER_dir = '/home/ecresswell/DCA_ER/'
    msa_npy_file = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir
if 0:
    DCA_ER_dir = '/data/cresswellclayec/DCA_ER/'
    msa_npy_file = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir




# Set DCA_ER directory
DCA_dir = os.getcwd()

# Define data directories
# Need to think on best way to do this..
# Referencing the same dataframe may be useful so we dont always have to load individual ref files...
individual_pdb_ref_file = Path(data_path, pfam_id, 'pdb_refs.npy')
pdb = np.load(individual_pdb_ref_file)
processed_data_dir = "%s/protein_data/data_processing_output" % DCA_dir

# delete 'b' in front of letters (python 2 --> python 3)
pdb = np.array([pdb[t,i].decode('UTF-8') for t in range(pdb.shape[0]) \
         for i in range(pdb.shape[1])]).reshape(pdb.shape[0],pdb.shape[1])


# Print number of pdb structures in Protein ID folder
npdb = pdb.shape[0]
print('number of pdb structures:',npdb)

# Create pandas dataframe for protein structure
pdb_df = pd.DataFrame(pdb,columns = ['PF','seq','id','uniprot_start','uniprot_end',\
                                 'pdb_id','chain','pdb_start','pdb_end'])
print(pdb_df.head())

# print("Direct Information from Expectation reflection:\n",di)
def no_diag(mat, diag_l, s_index=None, make_big=False):
    rows, columns = mat.shape
    if make_big:
        new_mat = 100. * np.ones((rows,columns))
    else:
        new_mat = np.zeros((rows,columns))
    for row in range(rows):
        for col in range(columns):
            if s_index is None:
                if abs(row-col) > diag_l:
                    new_mat[row, col] = mat[row ,col]
            else:
                if abs(s_index[row]-s_index[col]) > diag_l:
                    new_mat[row, col] = mat[row ,col]    
    return new_mat


number of pdb structures: 372
        PF seq            id uniprot_start uniprot_end pdb_id chain pdb_start  \
0  PF00186  69  Q5KZ26_GEOKA             1         160   1ZDR     B         1   
1  PF00186  69  Q5KZ26_GEOKA             1         160   1ZDR     A         1   
2  PF00186  83  Q81R22_BACAN             2         160   3JWK     B         2   
3  PF00186  83  Q81R22_BACAN             2         160   3S9U     B         2   
4  PF00186  83  Q81R22_BACAN             2         160   3FL9     H         2   

  pdb_end  
0     160  
1     160  
2     160  
3     160  
4     160  


In [3]:
import ecc_tools as tools
ipdb = 0
printing = True
print('seq:',int(pdb[ipdb,1]))



s0,cols_removed, s_index, tpdb, orig_seq_len = data_processing(data_path, pfam_id, ipdb,\
                gap_seqs=0.2, gap_cols=0.2, prob_low=0.004, conserved_cols=0.9, printing=printing, \
                                                               out_dir=processed_data_dir)
print('\n\n\nPreprocessed reference Sequence: ', s0[tpdb])

# npy2fa does not remove cols this way we are as close to original as possible
msa_outfile, ref_outfile, s0, cols_removed, s_index, tpdb, orig_seq_len  = tools.npy2fa(pfam_id, msa_npy_file,\
                                                                                        pdb_ref_file=pdb_ref_file,\
                                                                                        ipdb=ipdb, preprocess=True,\
                                                                                        gap_seqs=.2, gap_cols=.2, \
                                                                                        prob_low=.004, \
                                                                                        conserved_cols=.9, \
                                                                                        letter_format=False, \
                                                                                        first_10=True)

print('\n\nfirst 50 S: ', s0.shape, '\n\n')
pdb_id = pdb_df.iloc[ipdb]['pdb_id']
pdb_chain = pdb_df.iloc[ipdb]['chain']

pdb_pfam_map_file = Path('%s/protein_data/pdb_data/pdb_pfam_mapping.csv' % DCA_dir)
pdb_map_df = pd.read_csv(pdb_pfam_map_file, sep=',', header=1)
print(pdb_map_df.head())

pdb_id_map_df = pdb_map_df.loc[pdb_map_df['PDB']==pdb_id.lower()]
pdb_pfam_map = pdb_id_map_df.loc[pdb_id_map_df['CHAIN']==pdb_chain]


seq: 69
pdb ref example (pdb[0])  (after UTF-8 decode, removing 'b'):
 ['PF00186' '69' 'Q5KZ26_GEOKA' '1' '160' '1ZDR' 'B' '1' '160']
tpdb (s_ipdb) is :  69
#

-------------------------Remove Gaps--------------------------#
Shape of s is :  (7750, 918)
s = 
 [['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']
 ...
 ['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']]
s[tpdb] shape is  (160,)
s = 
 [['-' '-' '-' ... 'Y' 'E' 'K']
 ['M' 'I' 'S' ... 'W' 'E' 'R']
 ['-' 'L' 'A' ... 'Y' 'E' 'R']
 ...
 ['-' '-' '-' ... '-' '-' '-']
 ['-' 'V' 'S' ... 'Y' 'E' 'R']
 ['-' 'F' 'S' ... 'Y' 'E' 'K']]
though s still has gaps, s[69] does not:
 ['M' 'I' 'S' 'H' 'I' 'V' 'A' 'M' 'D' 'E' 'N' 'R' 'V' 'I' 'G' 'K' 'D' 'N'
 'R' 'L' 'P' 'W' 'H' 'L' 'P' 'A' 'D' 'L' 'A' 'Y' 'F' 'K' 'R' 'V' 'T' 'M'
 'G' 'H' 'A' 'I' 'V' 'M' 'G' 'R' 'K' 'T' 'F' 'E' 'A' 'I' 'G' 'R' 'P' 'L'
 'P' 'G' 'R' 'D' 'N' 'V' 'V' 'V' 'T' 'R' 'N' 'R' 'S' 'F' 'R' 'P' 

In [4]:
# Compute Frequencies printing everything out
from inference_dca import direct_info_dca
print(s0.shape)
seq_wt_file = None
seq_wt_file = '%s/protein_data/data_processing_output/seq_weight_%s.npy' % (DCA_ER_dir, pfam_id)

# ----------- DCA DI (MF) calculation --------------------------------------------- #

mf_di, fi, fij, c, cinv, w, w2d, fi_pydca, fij_pydca, c_pydca, c_inv_pydca, w_pydca, w2d_pydcak, di_pydca, ma_inv,seq_ints \
= direct_info_dca(s0, seq_wt_outfile=seq_wt_file, first10=True)
print('c[0]:',c[0])
print('c_pydca[0]', c_pydca[0])
def compute_sequences_weight(alignment_data=None, seqid=None):
    """Computes weight of sequences. The weights are calculated by lumping
    together sequences whose identity is greater that a particular threshold.
    For example, if there are m similar sequences, each of them will be assigned
    a weight of 1/m. Note that the effective number of sequences is the sum of
    these weights.

    Parameters
    ----------
        alignmnet_data : np.array()
            Numpy 2d array of the alignment data, after the alignment is put in
            integer representation
        seqid : float
            Value at which beyond this sequences are considered similar. Typical
            values could be 0.7, 0.8, 0.9 and so on

    Returns
    -------
        seqs_weight : np.array()
            A 1d numpy array containing computed weights. This array has a size
            of the number of sequences in the alignment data.
    """
    alignment_shape = alignment_data.shape
    num_seqs = alignment_shape[0]
    seqs_len = alignment_shape[1]
    seqs_weight = np.zeros((num_seqs,), dtype=np.float64)
    inv_seqs_weight = np.zeros((num_seqs,), dtype=np.float64)

    #count similar sequences
    for i in range(num_seqs):
        seq_i = alignment_data[i]
        for j in range(num_seqs):
            seq_j = alignment_data[j]
            iid = np.sum(seq_i==seq_j)
            if np.float64(iid)/np.float64(seqs_len) > seqid:
                seqs_weight[i] += 1
    #compute the weight of each sequence in the alignment
    for i in range(num_seqs): inv_seqs_weight[i] = 1.0/float(seqs_weight[i])
    np.save('first10_seq_weight_pydca.npy', seqs_weight)
    return seqs_weight, inv_seqs_weight


def compute_single_site_freqs(alignment_data=None,
        num_site_states=None, seqs_weight=None):
    """Computes single site frequency counts for a particular aligmnet data.

    Parameters
    ----------
        alignment_data : np.array()
            A 2d numpy array of alignment data represented in integer form.

        num_site_states : int
            An integer value fo the number of states a sequence site can have
            including a gap state. Typical value is 5 for RNAs and 21 for
            proteins.

        seqs_weight : np.array()
            A 1d numpy array of sequences weight

    Returns
    -------
        single_site_freqs : np.array()
            A 2d numpy array of of data type float64. The shape of this array is
            (seqs_len, num_site_states) where seqs_len is the length of sequences
            in the alignment data.
    """
    alignment_shape = alignment_data.shape
    #num_seqs = alignment_shape[0]
    seqs_len = alignment_shape[1]
    m_eff = np.sum(seqs_weight)
    single_site_freqs = np.zeros(shape = (seqs_len, num_site_states),
        dtype = np.float64)
    for i in range(seqs_len):
        
        # CHANGE
        #for a in range(1, num_site_states + 1):#we need gap states single site freqs too
        for a in range(num_site_states):#we need gap states single site freqs too

            column_i = alignment_data[:,i]
            freq_ia = np.sum((column_i==a)*seqs_weight)
            print('site %d-%d freq and count: ' % (i, a), freq_ia, np.sum((column_i==a)))
            single_site_freqs[i, a-1] = freq_ia/m_eff
    return single_site_freqs


def get_reg_single_site_freqs(single_site_freqs = None, seqs_len = None,
        num_site_states = None, pseudocount = None):
    """Regularizes single site frequencies.

    Parameters
    ----------
        single_site_freqs : np.array()
            A 2d numpy array of single site frequencies of shape
            (seqs_len, num_site_states). Note that gap state frequencies are
            included in this data.
        seqs_len : int
            The length of sequences in the alignment data
        num_site_states : int
            Total number of states that a site in a sequence can accommodate. It
            includes gap states.
        pseudocount : float
            This is the value of the relative pseudo count of type float.
            theta = lambda/(meff + lambda), where meff is the effective number of
            sequences and lambda is the real pseudo count.

    Returns
    -------
        reg_single_site_freqs : np.array()
            A 2d numpy array of shape (seqs_len, num_site_states) of single site
            frequencies after they are regularized.
    """
    reg_single_site_freqs = single_site_freqs
    theta_by_q = np.float64(pseudocount)/np.float64(num_site_states)
    for i in range(seqs_len):
        for a in range(num_site_states):
            reg_single_site_freqs[i, a] = theta_by_q + \
                (1.0 - pseudocount)*reg_single_site_freqs[i, a]
    return reg_single_site_freqs


def compute_pair_site_freqs(alignment_data=None, num_site_states=None, seqs_weight=None):
    """Computes pair site frequencies for an alignmnet data.

    Parameters
    ----------
        alignment_data : np.array()
            A 2d numpy array conatining alignment data. The residues in the
            alignment are in integer representation.
        num_site_states : int
            The number of possible states including gap state that sequence
            sites can accomodate. It must be an integer
        seqs_weight:
            A 1d numpy array of sequences weight

    Returns
    -------
        pair_site_freqs : np.array()
            A 3d numpy array of shape
            (num_pairs, num_site_states, num_site_states) where num_pairs is
            the number of unique pairs we can form from sequence sites. The
            pairs are assumed to in the order (0, 1), (0, 2) (0, 3), ...(0, L-1),
            ... (L-1, L). This ordering is critical and any change must be
            documented.
    """
    alignment_shape = alignment_data.shape
    num_seqs = alignment_shape[0]
    seqs_len = alignment_shape[1]
    num_site_pairs = (seqs_len -1)*seqs_len/2
    num_site_pairs = np.int64(num_site_pairs)
    m_eff = np.sum(seqs_weight)
    pair_site_freqs = np.zeros(
        shape=(num_site_pairs, num_site_states - 1, num_site_states - 1),
        dtype = np.float64
    )
    for i in range(seqs_len - 1):
        column_i = alignment_data[:, i]
        for j in range(i+1, seqs_len):
            pair_site = int((seqs_len * (seqs_len - 1)/2) - (seqs_len - i) * ((seqs_len - i) - 1)/2  + j  - i - 1)
            column_j = alignment_data[:, j]
            for a in range(1, num_site_states):
                count_ai = column_i==a
                for b in range(1, num_site_states):
                    count_bj = column_j==b
                    count_ai_bj = count_ai * count_bj
                    freq_ia_jb = np.sum(count_ai_bj*seqs_weight)
                    print('freq for %d-%d, %d-%d:' %(i,a,j,b), freq_ia_jb)
                    pair_site_freqs[pair_site, a-1, b-1] += freq_ia_jb/m_eff
    return pair_site_freqs


(6573, 10)
(6573, 10)
ma_inv (sequences weight shape:  (6573,)
site 0-0 freq and count: 0.0 0
site 0-1 freq and count: 0.0 0
site 0-2 freq and count: 0.0 0
site 0-3 freq and count: 0.0 0
site 0-4 freq and count: 0.0 0
site 0-5 freq and count: 0.0 0
site 0-6 freq and count: 0.0 0
site 0-7 freq and count: 0.0 0
site 0-8 freq and count: 0.0 0
site 0-9 freq and count: 0.0 0
site 0-10 freq and count: 1.0 1
site 0-11 freq and count: 0.0 0
site 0-12 freq and count: 0.0 0
site 0-13 freq and count: 0.0 0
site 0-14 freq and count: 0.0 0
site 0-15 freq and count: 0.0 0
site 0-16 freq and count: 0.0 0
site 0-17 freq and count: 0.0 0
site 0-18 freq and count: 0.0 0
site 0-19 freq and count: 0.0 0
site 0-20 freq and count: 1497.7235751716441 6572
site 1-0 freq and count: 0.0 0
site 1-1 freq and count: 0.0 0
site 1-2 freq and count: 0.0 0
site 1-3 freq and count: 0.0 0
site 1-4 freq and count: 61.1207123556262 159
site 1-5 freq and count: 0.0 0
site 1-6 freq and count: 0.0 0
site 1-7 freq and count: 

freq for 0-1, 4-8: 0.0
freq for 0-1, 4-9: 0.0
freq for 0-1, 4-10: 0.0
freq for 0-1, 4-11: 0.0
freq for 0-1, 4-12: 0.0
freq for 0-1, 4-13: 0.0
freq for 0-1, 4-14: 0.0
freq for 0-1, 4-15: 0.0
freq for 0-1, 4-16: 0.0
freq for 0-1, 4-17: 0.0
freq for 0-1, 4-18: 0.0
freq for 0-1, 4-19: 0.0
freq for 0-2, 4-0: 0.0
freq for 0-2, 4-1: 0.0
freq for 0-2, 4-2: 0.0
freq for 0-2, 4-3: 0.0
freq for 0-2, 4-4: 0.0
freq for 0-2, 4-5: 0.0
freq for 0-2, 4-6: 0.0
freq for 0-2, 4-7: 0.0
freq for 0-2, 4-8: 0.0
freq for 0-2, 4-9: 0.0
freq for 0-2, 4-10: 0.0
freq for 0-2, 4-11: 0.0
freq for 0-2, 4-12: 0.0
freq for 0-2, 4-13: 0.0
freq for 0-2, 4-14: 0.0
freq for 0-2, 4-15: 0.0
freq for 0-2, 4-16: 0.0
freq for 0-2, 4-17: 0.0
freq for 0-2, 4-18: 0.0
freq for 0-2, 4-19: 0.0
freq for 0-3, 4-0: 0.0
freq for 0-3, 4-1: 0.0
freq for 0-3, 4-2: 0.0
freq for 0-3, 4-3: 0.0
freq for 0-3, 4-4: 0.0
freq for 0-3, 4-5: 0.0
freq for 0-3, 4-6: 0.0
freq for 0-3, 4-7: 0.0
freq for 0-3, 4-8: 0.0
freq for 0-3, 4-9: 0.0
freq for 0-3, 

freq for 0-18, 7-0: 0.0
freq for 0-18, 7-1: 0.0
freq for 0-18, 7-2: 0.0
freq for 0-18, 7-3: 0.0
freq for 0-18, 7-4: 0.0
freq for 0-18, 7-5: 0.0
freq for 0-18, 7-6: 0.0
freq for 0-18, 7-7: 0.0
freq for 0-18, 7-8: 0.0
freq for 0-18, 7-9: 0.0
freq for 0-18, 7-10: 0.0
freq for 0-18, 7-11: 0.0
freq for 0-18, 7-12: 0.0
freq for 0-18, 7-13: 0.0
freq for 0-18, 7-14: 0.0
freq for 0-18, 7-15: 0.0
freq for 0-18, 7-16: 0.0
freq for 0-18, 7-17: 0.0
freq for 0-18, 7-18: 0.0
freq for 0-18, 7-19: 0.0
freq for 0-19, 7-0: 0.0
freq for 0-19, 7-1: 0.0
freq for 0-19, 7-2: 0.0
freq for 0-19, 7-3: 0.0
freq for 0-19, 7-4: 0.0
freq for 0-19, 7-5: 0.0
freq for 0-19, 7-6: 0.0
freq for 0-19, 7-7: 0.0
freq for 0-19, 7-8: 0.0
freq for 0-19, 7-9: 0.0
freq for 0-19, 7-10: 0.0
freq for 0-19, 7-11: 0.0
freq for 0-19, 7-12: 0.0
freq for 0-19, 7-13: 0.0
freq for 0-19, 7-14: 0.0
freq for 0-19, 7-15: 0.0
freq for 0-19, 7-16: 0.0
freq for 0-19, 7-17: 0.0
freq for 0-19, 7-18: 0.0
freq for 0-19, 7-19: 0.0
freq for 0-0, 8-0: 0

freq for 1-12, 3-19: 0.0
freq for 1-13, 3-0: 0.0
freq for 1-13, 3-1: 0.0
freq for 1-13, 3-2: 0.0
freq for 1-13, 3-3: 0.0
freq for 1-13, 3-4: 0.0
freq for 1-13, 3-5: 0.0
freq for 1-13, 3-6: 0.0
freq for 1-13, 3-7: 0.0
freq for 1-13, 3-8: 0.0
freq for 1-13, 3-9: 0.0
freq for 1-13, 3-10: 0.0
freq for 1-13, 3-11: 0.0
freq for 1-13, 3-12: 0.0
freq for 1-13, 3-13: 0.0
freq for 1-13, 3-14: 0.0
freq for 1-13, 3-15: 0.0
freq for 1-13, 3-16: 0.0
freq for 1-13, 3-17: 0.0
freq for 1-13, 3-18: 0.0
freq for 1-13, 3-19: 0.0
freq for 1-14, 3-0: 0.0
freq for 1-14, 3-1: 0.0
freq for 1-14, 3-2: 0.0
freq for 1-14, 3-3: 0.0
freq for 1-14, 3-4: 0.0
freq for 1-14, 3-5: 0.0
freq for 1-14, 3-6: 0.0
freq for 1-14, 3-7: 0.0
freq for 1-14, 3-8: 0.0
freq for 1-14, 3-9: 0.0
freq for 1-14, 3-10: 0.0
freq for 1-14, 3-11: 0.0
freq for 1-14, 3-12: 0.0
freq for 1-14, 3-13: 0.0
freq for 1-14, 3-14: 0.0
freq for 1-14, 3-15: 0.0
freq for 1-14, 3-16: 0.0
freq for 1-14, 3-17: 0.0
freq for 1-14, 3-18: 0.0
freq for 1-14, 3-19:

freq for 1-18, 6-8: 0.0
freq for 1-18, 6-9: 0.0
freq for 1-18, 6-10: 0.0
freq for 1-18, 6-11: 0.0
freq for 1-18, 6-12: 0.0
freq for 1-18, 6-13: 0.0
freq for 1-18, 6-14: 0.0
freq for 1-18, 6-15: 0.0
freq for 1-18, 6-16: 0.0
freq for 1-18, 6-17: 0.0
freq for 1-18, 6-18: 0.0
freq for 1-18, 6-19: 0.0
freq for 1-19, 6-0: 0.0
freq for 1-19, 6-1: 0.0
freq for 1-19, 6-2: 0.0
freq for 1-19, 6-3: 0.0
freq for 1-19, 6-4: 0.0
freq for 1-19, 6-5: 0.0
freq for 1-19, 6-6: 0.0
freq for 1-19, 6-7: 0.0
freq for 1-19, 6-8: 0.0
freq for 1-19, 6-9: 0.0
freq for 1-19, 6-10: 0.0
freq for 1-19, 6-11: 0.0
freq for 1-19, 6-12: 0.0
freq for 1-19, 6-13: 0.0
freq for 1-19, 6-14: 0.0
freq for 1-19, 6-15: 0.0
freq for 1-19, 6-16: 0.0
freq for 1-19, 6-17: 0.0
freq for 1-19, 6-18: 0.0
freq for 1-19, 6-19: 0.0
freq for 1-0, 7-0: 0.0
freq for 1-0, 7-1: 0.0
freq for 1-0, 7-2: 0.0
freq for 1-0, 7-3: 0.0
freq for 1-0, 7-4: 0.0
freq for 1-0, 7-5: 0.0
freq for 1-0, 7-6: 0.0
freq for 1-0, 7-7: 0.0
freq for 1-0, 7-8: 0.0
freq 

...

freq for 3-6, 9-3: 5.855555555555555
freq for 3-6, 9-4: 0.0
freq for 3-6, 9-5: 0.0
freq for 3-6, 9-6: 0.0
freq for 3-6, 9-7: 0.0
freq for 3-6, 9-8: 1.2058823529411764
freq for 3-6, 9-9: 1.0
freq for 3-6, 9-10: 0.0
freq for 3-6, 9-11: 2.5294117647058822
freq for 3-6, 9-12: 0.0
freq for 3-6, 9-13: 4.201687314492192
freq for 3-6, 9-14: 6.2785714285714285
freq for 3-6, 9-15: 1.2222222222222223
freq for 3-6, 9-16: 2.0
freq for 3-6, 9-17: 1.0
freq for 3-6, 9-18: 0.0
freq for 3-6, 9-19: 0.0
freq for 3-7, 9-0: 20.207754073500844
freq for 3-7, 9-1: 0.0
freq for 3-7, 9-2: 20.537185392044726
freq for 3-7, 9-3: 57.9702040947638
freq for 3-7, 9-4: 0.0
freq for 3-7, 9-5: 5.611111111111111
freq for 3-7, 9-6: 4.7351190476190474
freq for 3-7, 9-7: 3.5
freq for 3-7, 9-8: 59.005161221103805
freq for 3-7, 9-9: 11.305978260869566
freq for 3-7, 9-10: 1.5434782608695652
freq for 3-7, 9-11: 19.25116377583769
freq for 3-7, 9-12: 15.994973954494078
freq for 3-7, 9-13: 28.624809789105115
freq for 3-7, 9-14: 44.5

freq for 4-1, 8-10: 0.0
freq for 4-1, 8-11: 0.0
freq for 4-1, 8-12: 0.0
freq for 4-1, 8-13: 0.0
freq for 4-1, 8-14: 0.0
freq for 4-1, 8-15: 0.0
freq for 4-1, 8-16: 0.0
freq for 4-1, 8-17: 0.0
freq for 4-1, 8-18: 0.0
freq for 4-1, 8-19: 0.0
freq for 4-2, 8-0: 0.0
freq for 4-2, 8-1: 0.0
freq for 4-2, 8-2: 0.0
freq for 4-2, 8-3: 0.0
freq for 4-2, 8-4: 0.0
freq for 4-2, 8-5: 0.0
freq for 4-2, 8-6: 0.0
freq for 4-2, 8-7: 0.0
freq for 4-2, 8-8: 0.0
freq for 4-2, 8-9: 0.0
freq for 4-2, 8-10: 0.0
freq for 4-2, 8-11: 0.0
freq for 4-2, 8-12: 0.0
freq for 4-2, 8-13: 0.0
freq for 4-2, 8-14: 0.0
freq for 4-2, 8-15: 0.0
freq for 4-2, 8-16: 0.0
freq for 4-2, 8-17: 0.0
freq for 4-2, 8-18: 0.0
freq for 4-2, 8-19: 0.0
freq for 4-3, 8-0: 0.0
freq for 4-3, 8-1: 0.0
freq for 4-3, 8-2: 0.0
freq for 4-3, 8-3: 0.0
freq for 4-3, 8-4: 0.0
freq for 4-3, 8-5: 0.0
freq for 4-3, 8-6: 0.0
freq for 4-3, 8-7: 0.0
freq for 4-3, 8-8: 0.0
freq for 4-3, 8-9: 0.0
freq for 4-3, 8-10: 0.0
freq for 4-3, 8-11: 0.0
freq for 4-3

freq for 5-18, 7-15: 0.0
freq for 5-18, 7-16: 0.0
freq for 5-18, 7-17: 3.333333333333333
freq for 5-18, 7-18: 0.0
freq for 5-18, 7-19: 0.0
freq for 5-19, 7-0: 0.01694915254237288
freq for 5-19, 7-1: 1.0
freq for 5-19, 7-2: 0.0
freq for 5-19, 7-3: 1.0
freq for 5-19, 7-4: 0.0
freq for 5-19, 7-5: 0.0
freq for 5-19, 7-6: 0.0
freq for 5-19, 7-7: 0.0
freq for 5-19, 7-8: 0.0
freq for 5-19, 7-9: 0.0
freq for 5-19, 7-10: 0.42647058823529416
freq for 5-19, 7-11: 0.0
freq for 5-19, 7-12: 0.0
freq for 5-19, 7-13: 6.333333333333333
freq for 5-19, 7-14: 13.246405549614426
freq for 5-19, 7-15: 1.0
freq for 5-19, 7-16: 1.0
freq for 5-19, 7-17: 1.9861111111111112
freq for 5-19, 7-18: 0.0
freq for 5-19, 7-19: 0.0
freq for 5-0, 8-0: 153.85894532353296
freq for 5-0, 8-1: 14.956024531024532
freq for 5-0, 8-2: 87.61667663927543
freq for 5-0, 8-3: 0.0
freq for 5-0, 8-4: 0.0
freq for 5-0, 8-5: 62.0958889037842
freq for 5-0, 8-6: 0.0
freq for 5-0, 8-7: 0.0
freq for 5-0, 8-8: 0.0
freq for 5-0, 8-9: 0.0
freq for

freq for 6-0, 9-13: 0.0
freq for 6-0, 9-14: 0.0
freq for 6-0, 9-15: 0.0
freq for 6-0, 9-16: 0.0
freq for 6-0, 9-17: 0.0
freq for 6-0, 9-18: 0.0
freq for 6-0, 9-19: 0.0
freq for 6-1, 9-0: 0.0
freq for 6-1, 9-1: 0.0
freq for 6-1, 9-2: 0.0
freq for 6-1, 9-3: 0.0
freq for 6-1, 9-4: 0.0
freq for 6-1, 9-5: 0.0
freq for 6-1, 9-6: 0.0
freq for 6-1, 9-7: 0.0
freq for 6-1, 9-8: 0.0
freq for 6-1, 9-9: 0.0
freq for 6-1, 9-10: 0.0
freq for 6-1, 9-11: 0.0
freq for 6-1, 9-12: 0.0
freq for 6-1, 9-13: 0.0
freq for 6-1, 9-14: 0.0
freq for 6-1, 9-15: 0.0
freq for 6-1, 9-16: 0.0
freq for 6-1, 9-17: 0.0
freq for 6-1, 9-18: 0.0
freq for 6-1, 9-19: 0.0
freq for 6-2, 9-0: 0.0
freq for 6-2, 9-1: 0.0
freq for 6-2, 9-2: 0.0
freq for 6-2, 9-3: 0.0
freq for 6-2, 9-4: 0.0
freq for 6-2, 9-5: 0.0
freq for 6-2, 9-6: 0.0
freq for 6-2, 9-7: 0.0
freq for 6-2, 9-8: 0.0
freq for 6-2, 9-9: 0.0
freq for 6-2, 9-10: 0.0
freq for 6-2, 9-11: 0.0
freq for 6-2, 9-12: 0.0
freq for 6-2, 9-13: 0.0
freq for 6-2, 9-14: 0.0
freq for 6-2

freq for 8-18, 9-0: 0.0
freq for 8-18, 9-1: 0.0
freq for 8-18, 9-2: 0.0
freq for 8-18, 9-3: 0.0
freq for 8-18, 9-4: 0.0
freq for 8-18, 9-5: 0.0
freq for 8-18, 9-6: 0.0
freq for 8-18, 9-7: 0.0
freq for 8-18, 9-8: 0.0
freq for 8-18, 9-9: 0.0
freq for 8-18, 9-10: 0.0
freq for 8-18, 9-11: 0.0
freq for 8-18, 9-12: 0.0
freq for 8-18, 9-13: 0.0
freq for 8-18, 9-14: 0.0
freq for 8-18, 9-15: 0.0
freq for 8-18, 9-16: 0.0
freq for 8-18, 9-17: 0.0
freq for 8-18, 9-18: 0.0
freq for 8-18, 9-19: 0.0
freq for 8-19, 9-0: 0.0
freq for 8-19, 9-1: 0.0
freq for 8-19, 9-2: 0.0
freq for 8-19, 9-3: 0.0
freq for 8-19, 9-4: 0.0
freq for 8-19, 9-5: 0.0
freq for 8-19, 9-6: 0.0
freq for 8-19, 9-7: 0.0
freq for 8-19, 9-8: 0.0
freq for 8-19, 9-9: 0.0
freq for 8-19, 9-10: 0.0
freq for 8-19, 9-11: 0.0
freq for 8-19, 9-12: 0.0
freq for 8-19, 9-13: 0.0
freq for 8-19, 9-14: 0.0
freq for 8-19, 9-15: 0.0
freq for 8-19, 9-16: 0.0
freq for 8-19, 9-17: 0.0
freq for 8-19, 9-18: 0.0
freq for 8-19, 9-19: 0.0
meff for our MF =  1

In [11]:
theta = .2
#try:
 #   seq_weight = np.load('first10_seq_weight_pydca.npy')
#except(FileNotFoundError):
seq_weight, inv_seqs_weight = compute_sequences_weight(alignment_data=s0, seqid=float(1.-theta))
compute_single_site_freqs(alignment_data=s0, num_site_states=21, seqs_weight=seq_weight)


site 0-0 freq and count:  0.0 0
site 0-1 freq and count:  0.0 0
site 0-2 freq and count:  0.0 0
site 0-3 freq and count:  0.0 0
site 0-4 freq and count:  0.0 0
site 0-5 freq and count:  0.0 0
site 0-6 freq and count:  0.0 0
site 0-7 freq and count:  0.0 0
site 0-8 freq and count:  0.0 0
site 0-9 freq and count:  0.0 0
site 0-10 freq and count:  1.0 1
site 0-11 freq and count:  0.0 0
site 0-12 freq and count:  0.0 0
site 0-13 freq and count:  0.0 0
site 0-14 freq and count:  0.0 0
site 0-15 freq and count:  0.0 0
site 0-16 freq and count:  0.0 0
site 0-17 freq and count:  0.0 0
site 0-18 freq and count:  0.0 0
site 0-19 freq and count:  0.0 0
site 0-20 freq and count:  141508.0 6572
site 1-0 freq and count:  0.0 0
site 1-1 freq and count:  0.0 0
site 1-2 freq and count:  0.0 0
site 1-3 freq and count:  0.0 0
site 1-4 freq and count:  1205.0 159
site 1-5 freq and count:  0.0 0
site 1-6 freq and count:  0.0 0
site 1-7 freq and count:  60399.0 2630
site 1-8 freq and count:  0.0 0
site 1-9 

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 7.06668834e-06, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.99992933e-01,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.51535945e-03,
        0.00000000e+00, 0.00000000e+00, 4.26820909e-01, 0.00000000e+00,
        2.62902006e-01, 1.78080546e-03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        6.45965981e-02, 0.00000000e+00, 0.00000000e+00, 2.35384322e-01,
        0.00000000e+00],
       [2.94680904e-03, 0.00000000e+00, 2.17654001e-03, 5.03854878e-03,
        4.68450770e-02, 1.07342996e-02, 1.91507254e-02, 1.05858991e-02,
        0.00000000e+00, 0.00000000e+00, 1.16211690e-01, 0.00000000e+00,
        7.5125

In [12]:
compute_pair_site_freqs(alignment_data=s0, num_site_states=21, seqs_weight=seq_weight)

freq for 0-1, 1-1: 0.0
freq for 0-1, 1-2: 0.0
freq for 0-1, 1-3: 0.0
freq for 0-1, 1-4: 0.0
freq for 0-1, 1-5: 0.0
freq for 0-1, 1-6: 0.0
freq for 0-1, 1-7: 0.0
freq for 0-1, 1-8: 0.0
freq for 0-1, 1-9: 0.0
freq for 0-1, 1-10: 0.0
freq for 0-1, 1-11: 0.0
freq for 0-1, 1-12: 0.0
freq for 0-1, 1-13: 0.0
freq for 0-1, 1-14: 0.0
freq for 0-1, 1-15: 0.0
freq for 0-1, 1-16: 0.0
freq for 0-1, 1-17: 0.0
freq for 0-1, 1-18: 0.0
freq for 0-1, 1-19: 0.0
freq for 0-1, 1-20: 0.0
freq for 0-2, 1-1: 0.0
freq for 0-2, 1-2: 0.0
freq for 0-2, 1-3: 0.0
freq for 0-2, 1-4: 0.0
freq for 0-2, 1-5: 0.0
freq for 0-2, 1-6: 0.0
freq for 0-2, 1-7: 0.0
freq for 0-2, 1-8: 0.0
freq for 0-2, 1-9: 0.0
freq for 0-2, 1-10: 0.0
freq for 0-2, 1-11: 0.0
freq for 0-2, 1-12: 0.0
freq for 0-2, 1-13: 0.0
freq for 0-2, 1-14: 0.0
freq for 0-2, 1-15: 0.0
freq for 0-2, 1-16: 0.0
freq for 0-2, 1-17: 0.0
freq for 0-2, 1-18: 0.0
freq for 0-2, 1-19: 0.0
freq for 0-2, 1-20: 0.0
freq for 0-3, 1-1: 0.0
freq for 0-3, 1-2: 0.0
freq for 0-3

freq for 0-8, 2-18: 0.0
freq for 0-8, 2-19: 0.0
freq for 0-8, 2-20: 0.0
freq for 0-9, 2-1: 0.0
freq for 0-9, 2-2: 0.0
freq for 0-9, 2-3: 0.0
freq for 0-9, 2-4: 0.0
freq for 0-9, 2-5: 0.0
freq for 0-9, 2-6: 0.0
freq for 0-9, 2-7: 0.0
freq for 0-9, 2-8: 0.0
freq for 0-9, 2-9: 0.0
freq for 0-9, 2-10: 0.0
freq for 0-9, 2-11: 0.0
freq for 0-9, 2-12: 0.0
freq for 0-9, 2-13: 0.0
freq for 0-9, 2-14: 0.0
freq for 0-9, 2-15: 0.0
freq for 0-9, 2-16: 0.0
freq for 0-9, 2-17: 0.0
freq for 0-9, 2-18: 0.0
freq for 0-9, 2-19: 0.0
freq for 0-9, 2-20: 0.0
freq for 0-10, 2-1: 0.0
freq for 0-10, 2-2: 0.0
freq for 0-10, 2-3: 0.0
freq for 0-10, 2-4: 0.0
freq for 0-10, 2-5: 0.0
freq for 0-10, 2-6: 0.0
freq for 0-10, 2-7: 0.0
freq for 0-10, 2-8: 0.0
freq for 0-10, 2-9: 0.0
freq for 0-10, 2-10: 0.0
freq for 0-10, 2-11: 0.0
freq for 0-10, 2-12: 0.0
freq for 0-10, 2-13: 0.0
freq for 0-10, 2-14: 0.0
freq for 0-10, 2-15: 1.0
freq for 0-10, 2-16: 0.0
freq for 0-10, 2-17: 0.0
freq for 0-10, 2-18: 0.0
freq for 0-10, 2

freq for 0-4, 4-14: 0.0
freq for 0-4, 4-15: 0.0
freq for 0-4, 4-16: 0.0
freq for 0-4, 4-17: 0.0
freq for 0-4, 4-18: 0.0
freq for 0-4, 4-19: 0.0
freq for 0-4, 4-20: 0.0
freq for 0-5, 4-1: 0.0
freq for 0-5, 4-2: 0.0
freq for 0-5, 4-3: 0.0
freq for 0-5, 4-4: 0.0
freq for 0-5, 4-5: 0.0
freq for 0-5, 4-6: 0.0
freq for 0-5, 4-7: 0.0
freq for 0-5, 4-8: 0.0
freq for 0-5, 4-9: 0.0
freq for 0-5, 4-10: 0.0
freq for 0-5, 4-11: 0.0
freq for 0-5, 4-12: 0.0
freq for 0-5, 4-13: 0.0
freq for 0-5, 4-14: 0.0
freq for 0-5, 4-15: 0.0
freq for 0-5, 4-16: 0.0
freq for 0-5, 4-17: 0.0
freq for 0-5, 4-18: 0.0
freq for 0-5, 4-19: 0.0
freq for 0-5, 4-20: 0.0
freq for 0-6, 4-1: 0.0
freq for 0-6, 4-2: 0.0
freq for 0-6, 4-3: 0.0
freq for 0-6, 4-4: 0.0
freq for 0-6, 4-5: 0.0
freq for 0-6, 4-6: 0.0
freq for 0-6, 4-7: 0.0
freq for 0-6, 4-8: 0.0
freq for 0-6, 4-9: 0.0
freq for 0-6, 4-10: 0.0
freq for 0-6, 4-11: 0.0
freq for 0-6, 4-12: 0.0
freq for 0-6, 4-13: 0.0
freq for 0-6, 4-14: 0.0
freq for 0-6, 4-15: 0.0
freq for 0

freq for 0-9, 6-7: 0.0
freq for 0-9, 6-8: 0.0
freq for 0-9, 6-9: 0.0
freq for 0-9, 6-10: 0.0
freq for 0-9, 6-11: 0.0
freq for 0-9, 6-12: 0.0
freq for 0-9, 6-13: 0.0
freq for 0-9, 6-14: 0.0
freq for 0-9, 6-15: 0.0
freq for 0-9, 6-16: 0.0
freq for 0-9, 6-17: 0.0
freq for 0-9, 6-18: 0.0
freq for 0-9, 6-19: 0.0
freq for 0-9, 6-20: 0.0
freq for 0-10, 6-1: 0.0
freq for 0-10, 6-2: 0.0
freq for 0-10, 6-3: 0.0
freq for 0-10, 6-4: 0.0
freq for 0-10, 6-5: 0.0
freq for 0-10, 6-6: 0.0
freq for 0-10, 6-7: 0.0
freq for 0-10, 6-8: 0.0
freq for 0-10, 6-9: 0.0
freq for 0-10, 6-10: 0.0
freq for 0-10, 6-11: 0.0
freq for 0-10, 6-12: 0.0
freq for 0-10, 6-13: 0.0
freq for 0-10, 6-14: 0.0
freq for 0-10, 6-15: 0.0
freq for 0-10, 6-16: 0.0
freq for 0-10, 6-17: 0.0
freq for 0-10, 6-18: 0.0
freq for 0-10, 6-19: 0.0
freq for 0-10, 6-20: 0.0
freq for 0-11, 6-1: 0.0
freq for 0-11, 6-2: 0.0
freq for 0-11, 6-3: 0.0
freq for 0-11, 6-4: 0.0
freq for 0-11, 6-5: 0.0
freq for 0-11, 6-6: 0.0
freq for 0-11, 6-7: 0.0
freq for

freq for 0-18, 8-9: 0.0
freq for 0-18, 8-10: 0.0
freq for 0-18, 8-11: 0.0
freq for 0-18, 8-12: 0.0
freq for 0-18, 8-13: 0.0
freq for 0-18, 8-14: 0.0
freq for 0-18, 8-15: 0.0
freq for 0-18, 8-16: 0.0
freq for 0-18, 8-17: 0.0
freq for 0-18, 8-18: 0.0
freq for 0-18, 8-19: 0.0
freq for 0-18, 8-20: 0.0
freq for 0-19, 8-1: 0.0
freq for 0-19, 8-2: 0.0
freq for 0-19, 8-3: 0.0
freq for 0-19, 8-4: 0.0
freq for 0-19, 8-5: 0.0
freq for 0-19, 8-6: 0.0
freq for 0-19, 8-7: 0.0
freq for 0-19, 8-8: 0.0
freq for 0-19, 8-9: 0.0
freq for 0-19, 8-10: 0.0
freq for 0-19, 8-11: 0.0
freq for 0-19, 8-12: 0.0
freq for 0-19, 8-13: 0.0
freq for 0-19, 8-14: 0.0
freq for 0-19, 8-15: 0.0
freq for 0-19, 8-16: 0.0
freq for 0-19, 8-17: 0.0
freq for 0-19, 8-18: 0.0
freq for 0-19, 8-19: 0.0
freq for 0-19, 8-20: 0.0
freq for 0-20, 8-1: 1232.0
freq for 0-20, 8-2: 28791.0
freq for 0-20, 8-3: 0.0
freq for 0-20, 8-4: 0.0
freq for 0-20, 8-5: 9171.0
freq for 0-20, 8-6: 0.0
freq for 0-20, 8-7: 0.0
freq for 0-20, 8-8: 0.0
freq for

...

freq for 5-11, 7-20: 0.0
freq for 5-12, 7-1: 0.0
freq for 5-12, 7-2: 0.0
freq for 5-12, 7-3: 0.0
freq for 5-12, 7-4: 0.0
freq for 5-12, 7-5: 0.0
freq for 5-12, 7-6: 0.0
freq for 5-12, 7-7: 0.0
freq for 5-12, 7-8: 0.0
freq for 5-12, 7-9: 0.0
freq for 5-12, 7-10: 0.0
freq for 5-12, 7-11: 0.0
freq for 5-12, 7-12: 0.0
freq for 5-12, 7-13: 0.0
freq for 5-12, 7-14: 0.0
freq for 5-12, 7-15: 0.0
freq for 5-12, 7-16: 0.0
freq for 5-12, 7-17: 0.0
freq for 5-12, 7-18: 0.0
freq for 5-12, 7-19: 0.0
freq for 5-12, 7-20: 0.0
freq for 5-13, 7-1: 0.0
freq for 5-13, 7-2: 0.0
freq for 5-13, 7-3: 0.0
freq for 5-13, 7-4: 0.0
freq for 5-13, 7-5: 0.0
freq for 5-13, 7-6: 0.0
freq for 5-13, 7-7: 0.0
freq for 5-13, 7-8: 0.0
freq for 5-13, 7-9: 0.0
freq for 5-13, 7-10: 0.0
freq for 5-13, 7-11: 0.0
freq for 5-13, 7-12: 0.0
freq for 5-13, 7-13: 0.0
freq for 5-13, 7-14: 0.0
freq for 5-13, 7-15: 0.0
freq for 5-13, 7-16: 0.0
freq for 5-13, 7-17: 0.0
freq for 5-13, 7-18: 0.0
freq for 5-13, 7-19: 0.0
freq for 5-13, 7-2

freq for 6-5, 7-12: 0.0
freq for 6-5, 7-13: 0.0
freq for 6-5, 7-14: 0.0
freq for 6-5, 7-15: 0.0
freq for 6-5, 7-16: 0.0
freq for 6-5, 7-17: 0.0
freq for 6-5, 7-18: 0.0
freq for 6-5, 7-19: 0.0
freq for 6-5, 7-20: 0.0
freq for 6-6, 7-1: 0.0
freq for 6-6, 7-2: 0.0
freq for 6-6, 7-3: 0.0
freq for 6-6, 7-4: 0.0
freq for 6-6, 7-5: 0.0
freq for 6-6, 7-6: 0.0
freq for 6-6, 7-7: 0.0
freq for 6-6, 7-8: 0.0
freq for 6-6, 7-9: 0.0
freq for 6-6, 7-10: 0.0
freq for 6-6, 7-11: 0.0
freq for 6-6, 7-12: 0.0
freq for 6-6, 7-13: 0.0
freq for 6-6, 7-14: 0.0
freq for 6-6, 7-15: 0.0
freq for 6-6, 7-16: 0.0
freq for 6-6, 7-17: 0.0
freq for 6-6, 7-18: 0.0
freq for 6-6, 7-19: 0.0
freq for 6-6, 7-20: 0.0
freq for 6-7, 7-1: 0.0
freq for 6-7, 7-2: 0.0
freq for 6-7, 7-3: 0.0
freq for 6-7, 7-4: 0.0
freq for 6-7, 7-5: 0.0
freq for 6-7, 7-6: 0.0
freq for 6-7, 7-7: 0.0
freq for 6-7, 7-8: 0.0
freq for 6-7, 7-9: 0.0
freq for 6-7, 7-10: 0.0
freq for 6-7, 7-11: 0.0
freq for 6-7, 7-12: 0.0
freq for 6-7, 7-13: 0.0
freq for 6

freq for 6-12, 9-14: 0.0
freq for 6-12, 9-15: 0.0
freq for 6-12, 9-16: 0.0
freq for 6-12, 9-17: 0.0
freq for 6-12, 9-18: 0.0
freq for 6-12, 9-19: 0.0
freq for 6-12, 9-20: 0.0
freq for 6-13, 9-1: 0.0
freq for 6-13, 9-2: 0.0
freq for 6-13, 9-3: 0.0
freq for 6-13, 9-4: 0.0
freq for 6-13, 9-5: 0.0
freq for 6-13, 9-6: 0.0
freq for 6-13, 9-7: 0.0
freq for 6-13, 9-8: 0.0
freq for 6-13, 9-9: 0.0
freq for 6-13, 9-10: 0.0
freq for 6-13, 9-11: 0.0
freq for 6-13, 9-12: 0.0
freq for 6-13, 9-13: 0.0
freq for 6-13, 9-14: 0.0
freq for 6-13, 9-15: 0.0
freq for 6-13, 9-16: 0.0
freq for 6-13, 9-17: 0.0
freq for 6-13, 9-18: 0.0
freq for 6-13, 9-19: 0.0
freq for 6-13, 9-20: 0.0
freq for 6-14, 9-1: 0.0
freq for 6-14, 9-2: 0.0
freq for 6-14, 9-3: 0.0
freq for 6-14, 9-4: 0.0
freq for 6-14, 9-5: 0.0
freq for 6-14, 9-6: 0.0
freq for 6-14, 9-7: 0.0
freq for 6-14, 9-8: 0.0
freq for 6-14, 9-9: 0.0
freq for 6-14, 9-10: 0.0
freq for 6-14, 9-11: 0.0
freq for 6-14, 9-12: 0.0
freq for 6-14, 9-13: 0.0
freq for 6-14, 9-1

freq for 8-3, 9-13: 0.0
freq for 8-3, 9-14: 0.0
freq for 8-3, 9-15: 0.0
freq for 8-3, 9-16: 0.0
freq for 8-3, 9-17: 0.0
freq for 8-3, 9-18: 0.0
freq for 8-3, 9-19: 0.0
freq for 8-3, 9-20: 0.0
freq for 8-4, 9-1: 0.0
freq for 8-4, 9-2: 0.0
freq for 8-4, 9-3: 0.0
freq for 8-4, 9-4: 0.0
freq for 8-4, 9-5: 0.0
freq for 8-4, 9-6: 0.0
freq for 8-4, 9-7: 0.0
freq for 8-4, 9-8: 0.0
freq for 8-4, 9-9: 0.0
freq for 8-4, 9-10: 0.0
freq for 8-4, 9-11: 0.0
freq for 8-4, 9-12: 0.0
freq for 8-4, 9-13: 0.0
freq for 8-4, 9-14: 0.0
freq for 8-4, 9-15: 0.0
freq for 8-4, 9-16: 0.0
freq for 8-4, 9-17: 0.0
freq for 8-4, 9-18: 0.0
freq for 8-4, 9-19: 0.0
freq for 8-4, 9-20: 0.0
freq for 8-5, 9-1: 0.0
freq for 8-5, 9-2: 22.0
freq for 8-5, 9-3: 4143.0
freq for 8-5, 9-4: 0.0
freq for 8-5, 9-5: 0.0
freq for 8-5, 9-6: 30.0
freq for 8-5, 9-7: 12.0
freq for 8-5, 9-8: 3859.0
freq for 8-5, 9-9: 144.0
freq for 8-5, 9-10: 1.0
freq for 8-5, 9-11: 43.0
freq for 8-5, 9-12: 15.0
freq for 8-5, 9-13: 48.0
freq for 8-5, 9-14: 

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 2.35384322e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 

In [13]:
# Compute Frequencies printing everything out
from inference_dca import direct_info_dca
print(s0.shape)
seq_wt_file = None
seq_wt_file = '%s/protein_data/data_processing_output/first10_seq_weight_%s.npy' % (DCA_ER_dir, pfam_id)

# ----------- DCA DI (MF) calculation --------------------------------------------- #

mf_di, fi, fij, c, cinv, w, w2d, fi_pydca, fij_pydca, c_pydca, c_inv_pydca, w_pydca, w2d_pydcak, di_pydca, ma_inv, seq_ints \
= direct_info_dca(s0, seq_wt_outfile=seq_wt_file, first10=True)
print('c[0]:',c[0])
print('c_pydca[0]', c_pydca[0])

(6573, 10)
(6573, 10)
ma_inv (sequences weight shape:  (6573,)
site 0-0 freq and count: 0.0 0
site 0-1 freq and count: 0.0 0
site 0-2 freq and count: 0.0 0
site 0-3 freq and count: 0.0 0
site 0-4 freq and count: 0.0 0
site 0-5 freq and count: 0.0 0
site 0-6 freq and count: 0.0 0
site 0-7 freq and count: 0.0 0
site 0-8 freq and count: 0.0 0
site 0-9 freq and count: 0.0 0
site 0-10 freq and count: 1.0 1
site 0-11 freq and count: 0.0 0
site 0-12 freq and count: 0.0 0
site 0-13 freq and count: 0.0 0
site 0-14 freq and count: 0.0 0
site 0-15 freq and count: 0.0 0
site 0-16 freq and count: 0.0 0
site 0-17 freq and count: 0.0 0
site 0-18 freq and count: 0.0 0
site 0-19 freq and count: 0.0 0
site 0-20 freq and count: 1497.7235751716441 6572
site 1-0 freq and count: 0.0 0
site 1-1 freq and count: 0.0 0
site 1-2 freq and count: 0.0 0
site 1-3 freq and count: 0.0 0
site 1-4 freq and count: 61.1207123556262 159
site 1-5 freq and count: 0.0 0
site 1-6 freq and count: 0.0 0
site 1-7 freq and count: 

freq for 0-5, 3-12: 0.0
freq for 0-5, 3-13: 0.0
freq for 0-5, 3-14: 0.0
freq for 0-5, 3-15: 0.0
freq for 0-5, 3-16: 0.0
freq for 0-5, 3-17: 0.0
freq for 0-5, 3-18: 0.0
freq for 0-5, 3-19: 0.0
freq for 0-6, 3-0: 0.0
freq for 0-6, 3-1: 0.0
freq for 0-6, 3-2: 0.0
freq for 0-6, 3-3: 0.0
freq for 0-6, 3-4: 0.0
freq for 0-6, 3-5: 0.0
freq for 0-6, 3-6: 0.0
freq for 0-6, 3-7: 0.0
freq for 0-6, 3-8: 0.0
freq for 0-6, 3-9: 0.0
freq for 0-6, 3-10: 0.0
freq for 0-6, 3-11: 0.0
freq for 0-6, 3-12: 0.0
freq for 0-6, 3-13: 0.0
freq for 0-6, 3-14: 0.0
freq for 0-6, 3-15: 0.0
freq for 0-6, 3-16: 0.0
freq for 0-6, 3-17: 0.0
freq for 0-6, 3-18: 0.0
freq for 0-6, 3-19: 0.0
freq for 0-7, 3-0: 0.0
freq for 0-7, 3-1: 0.0
freq for 0-7, 3-2: 0.0
freq for 0-7, 3-3: 0.0
freq for 0-7, 3-4: 0.0
freq for 0-7, 3-5: 0.0
freq for 0-7, 3-6: 0.0
freq for 0-7, 3-7: 0.0
freq for 0-7, 3-8: 0.0
freq for 0-7, 3-9: 0.0
freq for 0-7, 3-10: 0.0
freq for 0-7, 3-11: 0.0
freq for 0-7, 3-12: 0.0
freq for 0-7, 3-13: 0.0
freq for 0-7

freq for 0-12, 5-4: 0.0
freq for 0-12, 5-5: 0.0
freq for 0-12, 5-6: 0.0
freq for 0-12, 5-7: 0.0
freq for 0-12, 5-8: 0.0
freq for 0-12, 5-9: 0.0
freq for 0-12, 5-10: 0.0
freq for 0-12, 5-11: 0.0
freq for 0-12, 5-12: 0.0
freq for 0-12, 5-13: 0.0
freq for 0-12, 5-14: 0.0
freq for 0-12, 5-15: 0.0
freq for 0-12, 5-16: 0.0
freq for 0-12, 5-17: 0.0
freq for 0-12, 5-18: 0.0
freq for 0-12, 5-19: 0.0
freq for 0-13, 5-0: 0.0
freq for 0-13, 5-1: 0.0
freq for 0-13, 5-2: 0.0
freq for 0-13, 5-3: 0.0
freq for 0-13, 5-4: 0.0
freq for 0-13, 5-5: 0.0
freq for 0-13, 5-6: 0.0
freq for 0-13, 5-7: 0.0
freq for 0-13, 5-8: 0.0
freq for 0-13, 5-9: 0.0
freq for 0-13, 5-10: 0.0
freq for 0-13, 5-11: 0.0
freq for 0-13, 5-12: 0.0
freq for 0-13, 5-13: 0.0
freq for 0-13, 5-14: 0.0
freq for 0-13, 5-15: 0.0
freq for 0-13, 5-16: 0.0
freq for 0-13, 5-17: 0.0
freq for 0-13, 5-18: 0.0
freq for 0-13, 5-19: 0.0
freq for 0-14, 5-0: 0.0
freq for 0-14, 5-1: 0.0
freq for 0-14, 5-2: 0.0
freq for 0-14, 5-3: 0.0
freq for 0-14, 5-4: 

freq for 0-6, 8-7: 0.0
freq for 0-6, 8-8: 0.0
freq for 0-6, 8-9: 0.0
freq for 0-6, 8-10: 0.0
freq for 0-6, 8-11: 0.0
freq for 0-6, 8-12: 0.0
freq for 0-6, 8-13: 0.0
freq for 0-6, 8-14: 0.0
freq for 0-6, 8-15: 0.0
freq for 0-6, 8-16: 0.0
freq for 0-6, 8-17: 0.0
freq for 0-6, 8-18: 0.0
freq for 0-6, 8-19: 0.0
freq for 0-7, 8-0: 0.0
freq for 0-7, 8-1: 0.0
freq for 0-7, 8-2: 0.0
freq for 0-7, 8-3: 0.0
freq for 0-7, 8-4: 0.0
freq for 0-7, 8-5: 0.0
freq for 0-7, 8-6: 0.0
freq for 0-7, 8-7: 0.0
freq for 0-7, 8-8: 0.0
freq for 0-7, 8-9: 0.0
freq for 0-7, 8-10: 0.0
freq for 0-7, 8-11: 0.0
freq for 0-7, 8-12: 0.0
freq for 0-7, 8-13: 0.0
freq for 0-7, 8-14: 0.0
freq for 0-7, 8-15: 0.0
freq for 0-7, 8-16: 0.0
freq for 0-7, 8-17: 0.0
freq for 0-7, 8-18: 0.0
freq for 0-7, 8-19: 0.0
freq for 0-8, 8-0: 0.0
freq for 0-8, 8-1: 0.0
freq for 0-8, 8-2: 0.0
freq for 0-8, 8-3: 0.0
freq for 0-8, 8-4: 0.0
freq for 0-8, 8-5: 0.0
freq for 0-8, 8-6: 0.0
freq for 0-8, 8-7: 0.0
freq for 0-8, 8-8: 0.0
freq for 0-8, 

freq for 1-2, 3-3: 0.0
freq for 1-2, 3-4: 0.0
freq for 1-2, 3-5: 0.0
freq for 1-2, 3-6: 0.0
freq for 1-2, 3-7: 0.0
freq for 1-2, 3-8: 0.0
freq for 1-2, 3-9: 0.0
freq for 1-2, 3-10: 0.0
freq for 1-2, 3-11: 0.0
freq for 1-2, 3-12: 0.0
freq for 1-2, 3-13: 0.0
freq for 1-2, 3-14: 0.0
freq for 1-2, 3-15: 0.0
freq for 1-2, 3-16: 0.0
freq for 1-2, 3-17: 0.0
freq for 1-2, 3-18: 0.0
freq for 1-2, 3-19: 0.0
freq for 1-3, 3-0: 0.0
freq for 1-3, 3-1: 0.0
freq for 1-3, 3-2: 0.0
freq for 1-3, 3-3: 0.0
freq for 1-3, 3-4: 0.0
freq for 1-3, 3-5: 0.0
freq for 1-3, 3-6: 0.0
freq for 1-3, 3-7: 0.0
freq for 1-3, 3-8: 0.0
freq for 1-3, 3-9: 0.0
freq for 1-3, 3-10: 0.0
freq for 1-3, 3-11: 0.0
freq for 1-3, 3-12: 0.0
freq for 1-3, 3-13: 0.0
freq for 1-3, 3-14: 0.0
freq for 1-3, 3-15: 0.0
freq for 1-3, 3-16: 0.0
freq for 1-3, 3-17: 0.0
freq for 1-3, 3-18: 0.0
freq for 1-3, 3-19: 0.0
freq for 1-4, 3-0: 6.783333333333333
freq for 1-4, 3-1: 2.8499999999999996
freq for 1-4, 3-2: 0.0
freq for 1-4, 3-3: 0.0
freq for

...

freq for 5-13, 6-5: 0.0
freq for 5-13, 6-6: 0.0
freq for 5-13, 6-7: 0.0
freq for 5-13, 6-8: 0.0
freq for 5-13, 6-9: 0.0
freq for 5-13, 6-10: 0.0
freq for 5-13, 6-11: 0.0
freq for 5-13, 6-12: 0.0
freq for 5-13, 6-13: 0.0
freq for 5-13, 6-14: 0.0
freq for 5-13, 6-15: 0.0
freq for 5-13, 6-16: 0.0
freq for 5-13, 6-17: 0.0
freq for 5-13, 6-18: 0.0
freq for 5-13, 6-19: 0.0
freq for 5-14, 6-0: 0.0
freq for 5-14, 6-1: 0.0
freq for 5-14, 6-2: 0.0
freq for 5-14, 6-3: 0.0
freq for 5-14, 6-4: 0.0
freq for 5-14, 6-5: 0.0
freq for 5-14, 6-6: 0.0
freq for 5-14, 6-7: 0.0
freq for 5-14, 6-8: 0.0
freq for 5-14, 6-9: 0.0
freq for 5-14, 6-10: 0.0
freq for 5-14, 6-11: 0.0
freq for 5-14, 6-12: 0.0
freq for 5-14, 6-13: 0.0
freq for 5-14, 6-14: 0.0
freq for 5-14, 6-15: 0.0
freq for 5-14, 6-16: 0.0
freq for 5-14, 6-17: 0.0
freq for 5-14, 6-18: 0.0
freq for 5-14, 6-19: 0.0
freq for 5-15, 6-0: 0.0
freq for 5-15, 6-1: 0.0
freq for 5-15, 6-2: 0.0
freq for 5-15, 6-3: 0.0
freq for 5-15, 6-4: 0.0
freq for 5-15, 6-5: 

freq for 5-11, 9-6: 0.0
freq for 5-11, 9-7: 0.0
freq for 5-11, 9-8: 0.0
freq for 5-11, 9-9: 0.0
freq for 5-11, 9-10: 0.0
freq for 5-11, 9-11: 0.0
freq for 5-11, 9-12: 0.0
freq for 5-11, 9-13: 0.0
freq for 5-11, 9-14: 0.0
freq for 5-11, 9-15: 0.0
freq for 5-11, 9-16: 0.0
freq for 5-11, 9-17: 0.0
freq for 5-11, 9-18: 0.0
freq for 5-11, 9-19: 0.0
freq for 5-12, 9-0: 0.0
freq for 5-12, 9-1: 0.0
freq for 5-12, 9-2: 0.0
freq for 5-12, 9-3: 0.0
freq for 5-12, 9-4: 0.0
freq for 5-12, 9-5: 0.0
freq for 5-12, 9-6: 0.0
freq for 5-12, 9-7: 0.0
freq for 5-12, 9-8: 0.0
freq for 5-12, 9-9: 0.0
freq for 5-12, 9-10: 0.0
freq for 5-12, 9-11: 0.0
freq for 5-12, 9-12: 0.0
freq for 5-12, 9-13: 0.0
freq for 5-12, 9-14: 0.0
freq for 5-12, 9-15: 0.0
freq for 5-12, 9-16: 0.0
freq for 5-12, 9-17: 0.0
freq for 5-12, 9-18: 0.0
freq for 5-12, 9-19: 0.0
freq for 5-13, 9-0: 0.0
freq for 5-13, 9-1: 0.0
freq for 5-13, 9-2: 0.0
freq for 5-13, 9-3: 0.0
freq for 5-13, 9-4: 0.0
freq for 5-13, 9-5: 0.0
freq for 5-13, 9-6: 

freq for 6-3, 9-5: 0.0
freq for 6-3, 9-6: 0.0
freq for 6-3, 9-7: 0.0
freq for 6-3, 9-8: 0.0
freq for 6-3, 9-9: 0.0
freq for 6-3, 9-10: 0.0
freq for 6-3, 9-11: 0.0
freq for 6-3, 9-12: 0.0
freq for 6-3, 9-13: 0.0
freq for 6-3, 9-14: 0.0
freq for 6-3, 9-15: 0.0
freq for 6-3, 9-16: 0.0
freq for 6-3, 9-17: 0.0
freq for 6-3, 9-18: 0.0
freq for 6-3, 9-19: 0.0
freq for 6-4, 9-0: 0.0
freq for 6-4, 9-1: 0.0
freq for 6-4, 9-2: 0.0
freq for 6-4, 9-3: 0.0
freq for 6-4, 9-4: 0.0
freq for 6-4, 9-5: 0.0
freq for 6-4, 9-6: 0.0
freq for 6-4, 9-7: 0.0
freq for 6-4, 9-8: 0.0
freq for 6-4, 9-9: 0.0
freq for 6-4, 9-10: 0.0
freq for 6-4, 9-11: 0.0
freq for 6-4, 9-12: 0.0
freq for 6-4, 9-13: 0.0
freq for 6-4, 9-14: 0.0
freq for 6-4, 9-15: 0.0
freq for 6-4, 9-16: 0.0
freq for 6-4, 9-17: 0.0
freq for 6-4, 9-18: 0.0
freq for 6-4, 9-19: 0.0
freq for 6-5, 9-0: 0.0
freq for 6-5, 9-1: 0.0
freq for 6-5, 9-2: 0.0
freq for 6-5, 9-3: 0.0
freq for 6-5, 9-4: 0.0
freq for 6-5, 9-5: 0.0
freq for 6-5, 9-6: 0.0
freq for 6-5, 

freq for 7-19, 9-11: 2.125
freq for 7-19, 9-12: 0.0
freq for 7-19, 9-13: 4.0
freq for 7-19, 9-14: 8.136904761904763
freq for 7-19, 9-15: 1.0
freq for 7-19, 9-16: 1.5714285714285712
freq for 7-19, 9-17: 1.0
freq for 7-19, 9-18: 0.0
freq for 7-19, 9-19: 0.0
freq for 8-0, 9-0: 49.83154442144627
freq for 8-0, 9-1: 0.0
freq for 8-0, 9-2: 48.50684418905096
freq for 8-0, 9-3: 80.16422248475804
freq for 8-0, 9-4: 0.0
freq for 8-0, 9-5: 8.211111111111112
freq for 8-0, 9-6: 14.017630649554153
freq for 8-0, 9-7: 2.128306878306878
freq for 8-0, 9-8: 55.789515887353325
freq for 8-0, 9-9: 6.340756302521008
freq for 8-0, 9-10: 1.8436452731665498
freq for 8-0, 9-11: 26.636407140060864
freq for 8-0, 9-12: 6.471899153478101
freq for 8-0, 9-13: 35.20524643995219
freq for 8-0, 9-14: 82.56647296564242
freq for 8-0, 9-15: 32.62052418794725
freq for 8-0, 9-16: 12.943828250138928
freq for 8-0, 9-17: 3.5442703749339906
freq for 8-0, 9-18: 0.0
freq for 8-0, 9-19: 0.0
freq for 8-1, 9-0: 0.3333333333333333
freq f

(10, 21)
(10, 21)
(45, 20, 20)
200
c[0]: [ 2.32426304e-02 -5.66893424e-04 -5.66893424e-04 -5.66893424e-04
 -5.66893424e-04 -5.66893424e-04 -5.66893424e-04 -5.66893424e-04
 -5.66893424e-04 -5.66893424e-04 -5.74836691e-04 -5.66893424e-04
 -5.66893424e-04 -5.66893424e-04 -5.66893424e-04 -5.66893424e-04
 -5.66893424e-04 -5.66893424e-04 -5.66893424e-04 -5.66893424e-04
  5.66893424e-04  5.66893424e-04  5.66893424e-04  5.66893424e-04
  8.13952707e-05  5.66893424e-04  5.66893424e-04 -3.56298748e-03
  5.66893424e-04 -2.43779522e-03  3.96553102e-04  5.66893424e-04
  5.66893424e-04  5.66893424e-04  5.66893424e-04  5.66893424e-04
  5.66893424e-04 -1.12747115e-03  5.66893424e-04  5.66893424e-04
 -7.66569410e-04  3.53642939e-04  5.66893424e-04  3.51167524e-04
  4.62703704e-04  6.55585857e-05  3.18012055e-04 -2.48375840e-04
  1.52454228e-06  5.66893424e-04  5.66893424e-04 -2.81084469e-04
  5.66893424e-04  3.70393604e-04  3.10108827e-04 -2.42718675e-03
 -9.62712119e-04 -2.18645151e-04  5.66893424e-04 

In [14]:
print('pydca sequences weight array length: ', seq_weight.shape)

pydca sequences weight array length:  (6573,)


In [15]:
print(ma_inv.shape)

(6573,)


In [16]:
print(seq_weight[:15])
print(seq_ints[:15])
print('\n\nthe difference between our seq_weight and their seq_weights for all %d sequences is %d.... QED\n\n' % 
      (s0.shape[0], abs(seq_weight - seq_ints).sum()))


print(inv_seqs_weight[:15])
print(ma_inv[:15])

[ 1.  6.  2. 98.  3. 34. 98. 11. 20. 22.  3.  4. 19. 24. 23.]
[ 1.  6.  2. 98.  3. 34. 98. 11. 20. 22.  3.  4. 19. 24. 23.]


the difference between our seq_weight and their seq_weights for all 6573 sequences is 0.... QED


[1.         0.16666667 0.5        0.01020408 0.33333333 0.02941176
 0.01020408 0.09090909 0.05       0.04545455 0.33333333 0.25
 0.05263158 0.04166667 0.04347826]
[1.         0.16666667 0.5        0.01020408 0.33333333 0.02941176
 0.01020408 0.09090909 0.05       0.04545455 0.33333333 0.25
 0.05263158 0.04166667 0.04347826]
